# Analysis of Arabidopsis genomes grown for 9 generations under temperature stress and control conditions

## Getting data

Data was generated and provided by the sequencing center of the University of Oslo.

## Quality check of fastq files

The quality check was carried out by the sequencing center and showed that all samples were of good enough quality for analysis. 

## Download of the reference genome 

```bash
#Download fasta file with information for all chromosomes   
wget https://www.arabidopsis.org/download_files/Genes/TAIR10_genome_release/TAIR10_chromosome_files/TAIR10_chr_all.fas

#Index all chomosome fasta files   
samtools faidx TAIR10_chr_all.fas
```

## Find the STR positions in the reference genome

```bash
#Download the Tandem Repeat Finder from this webpage: https://tandem.bu.edu/trf/trf409.macosx.download.html
#Generate dat file with STR positions using the Tandem Repeat Finder
./trf409.macosx TAIR10_chr_all.fas 2 7 7 80 10 30 9 -f -d -m -h

#rename HipsSTR output from TAIR10_chr_all.fas.2.7.7.80.10.30.500.dat to TAIR10_chr_all.fas.dat
mv TAIR10_chr_all.fas.2.7.7.80.10.30.9.dat TAIR10_chr_all.fas2.dat
```

To use the file in HipSTR it needs to be in bed file format so we convert it using the TRFdat_to_bed.py python script available here: https://github.com/hdashnow/TandemRepeatFinder_scripts/blob/master/TRFdat_to_bed.py

Make a text file and copy and paste the script from the website.

```bash 
#Convert dat file to bed file for HipSTR
./TRFdat_to_bed.py --dat TAIR10_chr_all.fas.dat --bed TAIR10_chr_all.fas.bed
./TRFdat_to_bed.py --dat TAIR10_chr_all.fas2.dat --bed TAIR10_chr_all.fas2.bed
```

```bash
#Check bed file
head -n 5 TAIR10_chr_all.fas.bed
head -n 5 TAIR10_chr_all.fas2.bed

#Index ref genome for bwa command to run
./../../../../../mnt/c/Users/greul/Downloads/bwa-master/bwa-master/bwa index TAIR10_chr_all.fas
bwa index TAIR10_chr_all.fas
```

Then split the bed file by chromosome so HipSTR can be run in parallel.

```r
#R
library("dplyr")

txtTair10 <- read.table("TAIR10_chr_all.fas2.bed", sep="\t")

annot_types <- c("1", "2", "3", "4", "5")

for (i in annot_types){ 
  for (j in annot_types){ 
    if (i == j){
      a <- filter(txtTair10, txtTair10[, 1] == i)
      write.table(a, file = paste0("trfStrPositionsTAIR10_", j, ".txt"), sep="\t", col.names=FALSE, row.names=FALSE, quote=FALSE)
    }
  }
}

head(a, n=10)
```

Rename the fastq files for easier use in bwa.  
The first command shows how it will look, the second command will actually rename the files.  
Remember to cut differently for different amount of sample number digits.  

```bash
#bash
for file in *; do
    name="${file%.*}"
    if [ ${#name} -eq 56 ]; then
       echo mv "$file" "$(echo "$file" | cut -c14-20,39-59)"
    fi
done

for file in *; do
    name="${file%.*}"
    if [ ${#name} -eq 56 ]; then
       cp "$file" "$(echo "$file" | cut -c14-20,39-59)"
    fi
done

```

There are some samples with the same library but run on different lanes resulting in two techincal replicates.
These samples need to be merged to only get one sample file for downstream analysis.

```bash

# Try with echo to see if correct file names are being inserted: 
find . -maxdepth 1 -iname "*_R1_001.fastq.gz" -type f | 
sed 's/_L00[78]\_R1_001.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   echo "cat ${f}_L007_R1_001.fastq.gz ${f}_L008_R1_001.fastq.gz > ${f}_R1_001.fastq.gz"
done

find . -maxdepth 1 -iname "*_R2_001.fastq.gz" -type f | 
sed 's/_L00[78]\_R2_001.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   echo "cat ${f}_L007_R2_001.fastq.gz ${f}_L008_R2_001.fastq.gz > ${f}_R2_001.fastq.gz"
done

# Then run actual command:
find . -maxdepth 1 -iname "*_R1_001.fastq.gz" -type f | 
sed 's/_L00[78]\_R1_001.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   cat ${f}_L007_R1_001.fastq.gz ${f}_L008_R1_001.fastq.gz > ${f}_R1_001.fastq.gz
done

find . -maxdepth 1 -iname "*_R2_001.fastq.gz" -type f | 
sed 's/_L00[78]\_R2_001.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   cat ${f}_L007_R2_001.fastq.gz ${f}_L008_R2_001.fastq.gz > ${f}_R2_001.fastq.gz
done

```

Then run bwa mem to map the fastq.gz files against the reference genome (takes ca. 1 hour per sample).

```bash 
# Try with echo to see if correct file names are being inserted: 
find . -maxdepth 1 -iname "*.fastq.gz" -type f | 
sed 's/_R[12]_001\.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   echo "bwa mem -t 20 ../TAIR10_chr_all.fas ${f}_R1_001.fastq.gz ${f}_R2_001.fastq.gz > ${f}.sam"
done

# Then run actual command:
find . -maxdepth 1 -iname "*.fastq.gz" -type f | 
sed 's/_R[12]_001\.fastq\.gz$//' | 
sort -u | 
while IFS= read -r f; do
   bwa mem -t 20 ../TAIR10_chr_all.fas ${f}_R1_001.fastq.gz ${f}_R2_001.fastq.gz > ${f}.sam
done
```

## Convert sam to bam

In order to use the files in HipSTR they need to be in bam file format.  
We convert them as follows:  

```bash 
#bash
#The module load is only necessary on SAGA
#module load samtools 
#module load parallel
# Try with echo to see if correct file names are being inserted:
find . -maxdepth 1 -iname "*.sam" -type f | 
sed 's/\.sam$//' | 
sort -u | 
while IFS= read -r f; do
   echo "samtools view -h -S -@ 20 -b ${f}.sam  > ${f}.bam"
done

# Then run actual command:
find . -maxdepth 1 -iname "*.sam" -type f | 
sed 's/\.sam$//' | 
sort -u | 
while IFS= read -r f; do
   samtools view -h -S -@ 20 -b ${f}.sam  > ${f}.bam 
done
```

Options for command *find*:  
-maxdepth *levels*: Descend at most levels (a non-negative integer) levels of directories below the starting-points.  *-maxdepth 0* means only apply the tests and actions to the starting-points themselves.

-iname *pattern*: Like -name, but the match is case insensitive. For example, the patterns 'fo*' and 'F??' match the file names Foo, FOO, foo, fOo, etc. The pattern 'foo' will also match a file called '.foobar'.

Check the bam file.

```bash 
# module load samtools
samtools view C-G0-S1_L007.bam | head
```

## Sort the bam files

Doing anything meaningful such as calling variants or visualizing alignments in (IGV) requires that the BAM is further manipulated. It must be sorted such that the alignments occur in “genome order”. That is, ordered positionally based upon their alignment coordinates on each chromosome. The -@20 flag can be added if more threads should be used when operating on SAGA. It does not work on PC.

```bash
module load samtools
# Try with echo to see if correct file names are being inserted:
find . -maxdepth 1 -iname "*.bam" -type f | 
sed 's/\.bam$//' | 
sort -u | 
while IFS= read -r f; do
   echo "samtools sort ${f}.bam -o ${f}.sorted.bam -@ 20"
done

# Then run actual command:
find . -maxdepth 1 -iname "*.bam" -type f | 
sed 's/\.bam$//' | 
sort -u | 
while IFS= read -r f; do
   samtools sort -@ 20 ${f}.bam -o ${f}.sorted.bam 
done
```

Check the sorted bam files and move them to the bams folder.

```bash 
samtools view C-G0-S1_L007.sorted.bam | head
mkdir bams
mv C-*.sorted.bam ../bams
```

## Index the sorted bam files

Index the sorted bam files so they can be used in HipSTR and copy to SAGA bams folder.

```bash 
cd bams
# module load samtools
# Try with echo to see if correct file names are being inserted:
find . -maxdepth 1 -iname "*.sorted.bam" -type f | 
sed 's/\.sorted\.bam$//' | 
sort -u | 
while IFS= read -r f; do
   echo "samtools index -b ${f}.sorted.bam"
done

# Then run actual command:
find . -maxdepth 1 -iname "*.sorted.bam" -type f | 
sed 's/\.sorted\.bam$//' | 
sort -u | 
while IFS= read -r f; do
   samtools index -b ${f}.sorted.bam
done

scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/bams/* annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra/bams
```

Copy the fasta file and the STR region files to SAGA.

```bash 
# mac
scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/TAIR10_chr_all.fas annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra

scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/trfStrPositionsTAIR10_[12345]* annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra

# windows
scp /mnt/e/PhD/ArabidopsisGenerationAnalysis/analysis/TAIR10_chr_all.fas  annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra

scp /mnt/f/PhD/ArabidopsisGenerationAnalysis/analysis/trfStrPositionsTAIR10_[12345]*  annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra
```

## Running HipSTR - Haplotype inference and phasing for Short Tandem Repeats

To compare short tandem repeats from the sequencing data to the reference genome we run HipSTR. 

Save the following script as a sh file using a text editor and upload the file to SAGA.

```bash 
#!/bin/sh
#SBATCH --job-name=hipstrArabidopsisGenerationAnalysis
#SBATCH --account=nn9244k
#SBATCH --time=168:0:0
#SBATCH --mem-per-cpu=30000M 
#SBATCH --partition=bigmem 
#SBATCH --cpus-per-task=5
#SBATCH --mail-user=annegreu@ibv.uio.no
#SBATCH --mail-type=ALL

#Run HipSTR
ALIGNMENTS=/cluster/work/users/annegreu/genSeqAra/bams
echo $ALIGNMENTS
HIPSTR_OUT=/cluster/work/users/annegreu/genSeqAra/HipSTR
echo $HIPSTR_OUT
BAMS=$(ls $ALIGNMENTS/*.bam | paste -sd, -)
echo $BAMS
BAM_SAMPLES=$(ls $ALIGNMENTS/*.bam | cut -d '/' -f8 | cut -d '.' -f1 | paste -sd, -)
echo $BAM_SAMPLES

#Load in modules on SAGA
#module --force purge
module load StdEnv
module load HipSTR/0.6.2-foss-2018b-Python-3.6.6
module swap GCCcore/7.3.0 GCCcore/8.3.0
module swap zlib/1.2.11-GCCcore-7.3.0 zlib/1.2.11-GCCcore-8.3.0
module load parallel/20190922-GCCcore-8.3.0
#module swap GCCcore/8.3.0 GCCcore/7.3.0 , GCCcore needed for HipSTR, is default so by doing purge should be set automatically.

#For parallel run by chromosome
parallel /cluster/software/HipSTR/0.6.2-foss-2018b-Python-3.6.6/bin/HipSTR\
 --bams $BAMS\
 --fasta /cluster/work/users/annegreu/genSeqAra/TAIR10_chr_all.fas\
 --regions /cluster/work/users/annegreu/genSeqAra/trfStrPositionsTAIR10_{}.txt\
 --str-vcf $HIPSTR_OUT/genSeqAra_LG{}.hipstr.vcf.gz\
 --log /cluster/work/users/annegreu/genSeqAra/genSeqAra.hipstr.log\
 --bam-samps $BAM_SAMPLES\
 --bam-libs $BAM_SAMPLES\
 --def-stutter-model\
 --max-haps 100 ::: {1..5}
 
#Test single chromosome
/cluster/software/HipSTR/0.6.2-foss-2018b-Python-3.6.6/bin/HipSTR\
 --bams $BAMS\
 --fasta /cluster/work/users/annegreu/genSeqAra/gadMor2.fasta\
 --regions /cluster/work/users/annegreu/genSeqAra/trfStrPositionsGadmor2_LG01.txt\
 --str-vcf $HIPSTR_OUT/testing.hipstr.vcf.gz\
 --log /cluster/work/users/annegreu/genSeqAra/codWildNofimaTest.hipstr.log\
 --bam-samps $BAM_SAMPLES\
 --bam-libs $BAM_SAMPLES\
 --def-stutter-model\
 --max-haps 100 
```

```bash 
#Upload HipSTR script to SAGA to run.
# mac
scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/genSeqAraHipstr.sh annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra

scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/runHipstr.sh annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra/bams2

# windows
scp /mnt/e/PhD/ArabidopsisGenerationAnalysis/analysis/genSeqAraHipstr.sh annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra
```

```bash
#Send job into queue
sbatch genSeqAraHipstr.sh 

#Check status of job
squeue -u annegreu
```

If the HipSTR was run for individual chromosomes in parallel the vcf files need to be merged.

```bash 
#!/bin/sh
#SBATCH --job-name=genSeqAraMerge
#SBATCH --account=nn9244k
#SBATCH --time=2:0:0
#SBATCH --mem-per-cpu=4GB  
#SBATCH --cpus-per-task=5
#SBATCH --mail-user=annegreu@ibv.uio.no
#SBATCH --mail-type=ALL

module load  VCFtools/0.1.16-foss-2018b-Perl-5.28.0

vcf-concat genSeqAra_LG1.hipstr.vcf.gz genSeqAra_LG2.hipstr.vcf.gz genSeqAra_LG3.hipstr.vcf.gz genSeqAra_LG4.hipstr.vcf.gz genSeqAra_LG5.hipstr.vcf.gz | gzip -c > genSeqAra.hipstr.vcf.gz
```

```bash
#Copy SLURM script to SAGA
scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/concatVcfs.sh annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra

#Send job into queue
sbatch concatVcfs.sh 

#Check status of job
squeue -u annegreu
```

Copy the HipSTR output (vcf file) to your own computer or harddrive as backup. Also copy the individual files.

```bash 
scp annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra/bams2/gen* /Volumes/SeagateNo1/PhD/ArabidopsisGenerationAnalysis/analysis/analysisNineGen
```

Copy the renamed files to SAGA

```bash
scp /Volumes/Seagate/PhD/ArabidopsisGenerationAnalysis/analysis/analysisData2/* annegreu@saga.sigma2.no:/cluster/work/users/annegreu/genSeqAra/fastqFiles

```